# Colab Demo Code

The demo code is inspired from the XMem Colab Demo code

In [ ]:
#

In [1]:
!nvidia-smi

import torch

if torch.cuda.is_available():
  print('Using GPU')
  device = 'cuda'
else:
  print('CUDA not available. Please connect to a GPU instance if possible.')
  device = 'cpu'

zsh:1: command not found: nvidia-smi
CUDA not available. Please connect to a GPU instance if possible.


In [2]:
%pip install -r requirements.txt

  Using cached ultralytics-8.3.26-py3-none-any.whl.metadata (35 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached numpy-2.1.2-cp313-cp313-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached matplotlib-3.9.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached pillow-11.0.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.1 kB)
  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached scipy-1.14.1-cp313-cp313-macosx_14_0_arm64.whl.metadata (60 kB)
INFO: pip is looking at multiple versions of ultralytics to determine which version is compatible with other requirements. This could take a while.
  Using cached ultralytics-8.3.25-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytics-8.3.24-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytics-8.3.23-py3-none-any.whl.metadata (35 kB)
  Using cached ultralytic

# Get XMem and install prerequisit

In [ ]:
!git clone https://github.com/hkchengrex/XMem.git
%cd XMem
%pip install opencv-python
%pip install matplotlib
%pip install -U numpy
%pip install -r requirements.txt

In [6]:
!git clone https://github.com/hkchengrex/XMem.git
%cd XMem
%pip install -r requirements.txt

Cloning into 'XMem'...
remote: Enumerating objects: 608, done.
remote: Counting objects: 100% (350/350), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 608 (delta 257), reused 215 (delta 215), pack-reused 258 (from 1)
Receiving objects: 100% (608/608), 263.42 KiB | 1.04 MiB/s, done.
Resolving deltas: 100% (359/359), done.
/Users/jonasjostan/Documents/Bachelorarbeit/Leprosy/XMem


/Users/jonasjostan/Documents/Bachelorarbeit/Leprosy/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


  Cloning https://github.com/cheind/py-thin-plate-spline to /private/var/folders/c2/cdv0qsp57bjbb3ww0lrp80pm0000gn/T/pip-req-build-en53k3mv
  Running command git clone --filter=blob:none --quiet https://github.com/cheind/py-thin-plate-spline /private/var/folders/c2/cdv0qsp57bjbb3ww0lrp80pm0000gn/T/pip-req-build-en53k3mv
  Resolved https://github.com/cheind/py-thin-plate-spline to commit f6995795397118b7d0ac01aecd3f39ffbfad9dee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 5.7 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for thinplate: filename=thinplate-1.0.0-py3-none-any.whl size=6703 sha256=d960053df916003ac7c345

# Basic setup


In [2]:
%cd XMem

/Users/jonasjostan/Documents/Bachelorarbeit/Leprosy/XMem


/Users/jonasjostan/Documents/Bachelorarbeit/Leprosy/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
%matplotlib inline

In [ ]:
# Insert here the path to the weights of YOLOv8
path_weights_YOLO_detection = ""
path_weights_YOLO_segmentation = ""
xmem_path_weights = ""

# Insert here the path to the data
data_path = ""

In [4]:
import os
from os import path
from argparse import ArgumentParser
import shutil

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

from inference.data.test_datasets import LongTestDataset, DAVISTestDataset, YouTubeVOSTestDataset
from inference.data.mask_mapper import MaskMapper
from model.network import XMem
from inference.inference_core import InferenceCore

from progressbar import progressbar

torch.set_grad_enabled(False)

# default configuration
xmem_config = {
    'top_k': 30,
    'mem_every': 5,
    'deep_update_every': -1,
    'enable_long_term': True,
    'enable_long_term_count_usage': True,
    'num_prototypes': 128,
    'min_mid_term_frames': 5,
    'max_mid_term_frames': 10,
    'max_long_term_elements': 10000,
}


network = XMem(xmem_config, xmem_path_weights).eval().to(device)

/Users/jonasjostan/Documents/Bachelorarbeit/Leprosy/XMem/model/network.py:145: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_weights = torch.load(model_path, map_locat

FileNotFoundError: [Errno 2] No such file or directory: './saves/XMem.pth'

Configuration for YOLOv8. For all options, please refer to https://docs.ultralytics.com/modes/predict/#inference-arguments

In [ ]:
yolo_config = {
    'conf': 0.70,
    'max_det': 1,
    'show': False,
}

yolo_seg_config = {
    'conf': 0.25,
    'max_det': 1,
    'bounding_box': False, 
    'show': False
}

In [ ]:
import ultralytics

# Detecting the key frames with YOLOv8

In [ ]:
from ultralytics import YOLO

model = YOLO(path_weights_YOLO_detection)
prediction = model(data_path, max_det=yolo_config['max_det'],conf=yolo_config['conf'], show=yolo_config['show'])
image = prediction['orig_img']


# Segment the images

In [5]:
from ultralytics import YOLO

model = YOLO(path_weights_YOLO_segmentation)
prediction = model(data_path, max_det=yolo_seg_config['max_det'],conf=yolo_seg_config['conf'], show=yolo_seg_config['show'])
image = prediction['orig_img']

NameError: name 'path_weights' is not defined

# Convert the YOLO output to a binary segmentation mask

The function needs the original image to create the new image

In [4]:
import cv2
import os

def denormalize_point(x:float, y:float, width:int, height:int):
    return int(x*width), int(y*height)

# Read contours from text file
def convert_contours_from_result(contour_normalized, width, height):
    contours = []
    for line in contour_normalized:
        points = line.strip().split(' ')
        contour = [(denormalize_point(float(points[i]), float(points[i+1]),width,height)) for i in range(1, len(points), 2)]
        contours.append(contour)
    return contours

# Create a blank image
def create_blank_image(width, height):
    return np.zeros((height, width), dtype=np.uint8)

# Draw filled contours on image
def draw_filled_contours(image, contours):
    for contour in contours:
        pts = np.array(contour, dtype=np.int32)
        cv2.fillPoly(image, [pts], 1)

# Save image
def save_image(image, file_path):
    cv2.imwrite(file_path, image)

def get_shape(image_path:str):
    shape = cv2.imread(image_path).shape
    height = shape[0]
    width = shape[1]
    return width, height

def convert_to_binary_segmentation_mask(original_width:int,original_height:int, contoursfile:str, output_file:str):
    # width, height = get_shape(original_image_path)
    width, height = original_width, original_height
    contours = convert_contours_from_result(contoursfile, width=width,height=height)
    
    blank_image = create_blank_image(width, height)
    draw_filled_contours(blank_image, contours)
    return blank_image.np()


In [ ]:


mask = convert_to_binary_segmentation_mask(prediction)

# Propagate the remaining frames

In [ ]:
import cv2
from inference.interact.interactive_utils import image_to_torch, index_numpy_to_one_hot_torch, torch_prob_to_numpy_mask, overlay_davis

torch.cuda.empty_cache()

processor = InferenceCore(network, config=xmem_config)
processor.set_all_labels(range(1, num_objects+1)) # consecutive labels
cap = cv2.VideoCapture(video_name)

# You can change these two numbers
frames_to_propagate = 417
visualize_every = 50

current_frame_index = 0

with torch.cuda.amp.autocast(enabled=True):
  while (cap.isOpened()):
    # load frame-by-frame
    _, frame = cap.read()
    if frame is None or current_frame_index > frames_to_propagate:
      break

    # convert numpy array to pytorch tensor format
    frame_torch, _ = image_to_torch(frame, device=device)
    if current_frame_index == 0:
      # initialize with the mask
      mask_torch = index_numpy_to_one_hot_torch(mask, num_objects+1).to(device)
      # the background mask is not fed into the model
      prediction = processor.step(frame_torch, mask_torch[1:])

    else:
      # propagate only
      prediction = processor.step(frame_torch)

    # argmax, convert to numpy
    prediction = torch_prob_to_numpy_mask(prediction)

    # TODO save the image here, if wanted 


    if current_frame_index % visualize_every == 0:
      print(current_frame_index)
      visualization = overlay_davis(frame, prediction)
      display(Image.fromarray(visualization))

    current_frame_index += 1